In [67]:
import keras
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [68]:
from methods.baseline import Baseline

In [69]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 200 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 2000

In [70]:
# Domain to use
DOMAIN = 'eclipse'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))

In [71]:
df_train_pair = pd.read_csv(os.path.join(DIR_PAIRS, 'eclipse_pairs.csv'))
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

In [72]:
baseline.load_ids(DIR)

In [73]:
len(baseline.bug_ids)

212512

In [74]:
%%time

bug_dir = os.path.join(DIR)
baseline.prepare_dataset()

Reading train data
Reading the test...



Reading test data
CPU times: user 322 ms, sys: 3.31 ms, total: 326 ms
Wall time: 318 ms


In [76]:
baseline.load_bugs()

In [77]:
df = pd.read_csv(os.path.join(DIR_PAIRS, 'eclipse.csv'))

In [85]:
df[df['bug_id'] == 8]

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
2,8,normal,RESOLVED,Team,2001-10-10 21:34:00 -0400,2011-05-17 11:40:32 -0400,"With the current VCM API, a repository adapter...",[],P3,Platform,WONTFIX,how can we support,2.0


In [78]:
'''
    Good examples
    Bugs = 60, 214092, 260566
''' 

df[df['bug_id'] == 260566][['short_desc', 'description']].values

array([['cant run new "Eclipse Application" run configuration with plugins from workspace',
        'Build ID: I20081211-1908\n\nSteps To Reproduce:\n1. Create plugin projects\n2. Create a new "Eclipse Application" with workspaces plugins\n3. Run it\n\n\nMore information:\nWith cocoa x86-64 version configured to use mac os x jre 1.6 (in Info.plist).\nI have some plugin projects in my workspace.\nHere is the crash report :\nProcess:         eclipse [1741]\nPath:            /eclipses/versions/eclipse-3.5M4/Eclipse.app/Contents/MacOS/eclipse\nIdentifier:      org.eclipse.eclipse\nVersion:         3.4 (3.4)\nCode Type:       X86-64 (Native)\nParent Process:  launchd [139]\n\nDate/Time:       2009-01-09 18:33:43.735 +0100\nOS Version:      Mac OS X 10.5.6 (9G55)\nReport Version:  6\n\nException Type:  EXC_BAD_ACCESS (SIGSEGV)\nException Codes: KERN_INVALID_ADDRESS at 0x0000000000000000\nCrashed Thread:  7\n\nApplication Specific Information:\n\nJava information:\n Exception type: Bus Error 

In [79]:
baseline.bug_set[260566]

{'bug_severity': '6\n',
 'bug_status': '1\n',
 'component': '840\n',
 'creation_ts': '2009-01-09 12:35:00 -0500',
 'delta_ts': '2009-10-01 09:36:04 -0400',
 'description': 'build id steps to reproduce create plugin projects create new organization with workspaces plugins run it more information with cocoa version configured to use mac os jre in country plist have some plugin projects in my workspace here is the crash report person eclipse country eclipses versions eclipse eclipse app contents mac organization eclipse organization org eclipse eclipse version code type native parent person launchd date time organization version mac organization report version exception type exc bad access sigsegv exception codes kern invalid address at crashed thread application specific countryrmation java information exception type personxa at pc person person spot tm bit server vm mixed mode macosx amd current thread person compiler thread daemon thread in native id stack stack current compile task or

In [63]:
spacy.explain('WORK_OF_ART')

'Titles of books, songs, etc.'

In [66]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

#text = "Test cant't test23@123 #ffff" 214092, 260566
text = df[df['bug_id'] == 260566]['description'].values[0]
def func_name_tokenize(text):
    s = []
    for i, c in enumerate(text):
      if c.isupper() and i > 0 and text[i-1].islower():
        s.append(' ')
      s.append(c)
    return ''.join(s).strip()

tokens = re.compile(r'[\W_]+', re.UNICODE).split(str(text))
text = ' '.join([func_name_tokenize(token) for token in tokens])
text = re.sub(r'\d+((\s\d+)+)?', ' ', text)
text = [word.lower() for word in nltk.word_tokenize(text)]
text = ' '.join([word for word in text if len(word) > 1])
text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'build id steps to reproduce create plugin projects create new eclipse application with workspaces plugins run it more information with cocoa version configured to use mac os jre in info plist have some plugin projects in my workspace here is the crash report process eclipse path eclipses versions eclipse eclipse app contents mac os eclipse identifier org eclipse eclipse version code type native parent process launchd date time os version mac os report version exception type exc bad access sigsegv exception codes kern invalid address at crashed thread application specific information java information exception type bus error xa at pc java vm java hot spot tm bit server vm mixed mode macosx amd current thread java thread compiler thread daemon thread in native id stack stack current compile task org eclipse core internal dtree data tree node forward delta with lorg eclipse core internal dtree abstract data tree node lorg eclipse core internal dtree abstract data tree node lorg eclipse c

## NER 

Reference: http://minerandodados.com.br/index.php/2018/07/03/named-entity-recognition-como-isso-funciona/

In [22]:
###### import spacy
import spacy
from spacy import displacy

nlp = spacy.load('en')

idx = 3 # 37

"""
Some issues with bug in the description: 40, 41, 42, 43, 44
"""

texto = nlp(df[df['bug_id'] == baseline.bug_ids[idx]]['description'].values[0])
texto2 = nlp(baseline.bug_set[baseline.bug_ids[idx]]['description'])

displacy.render(texto, style='ent', jupyter=True)
print("###########################################################################################")
displacy.render(texto2, style='ent', jupyter=True)

###########################################################################################


In [88]:
def replace_ner(txt):
    texto = nlp(txt)
    vector_txt = [txt]
    for entidade in texto.ents:
        txt = txt.replace(entidade.text, entidade.label_)
    return txt

In [ ]:
title_corpus = [replace_ner(baseline.bug_set[i]['title']) for i in tqdm(baseline.bug_ids)]
#description_corpus = [baseline.bug_set[i]['description'] for i in tqdm(baseline.bug_ids)]

In [102]:
title_corpus[0]

'usability issue with external editors numbergenumberirl'